# Performance for airlines problem
As we saw earlier, when we ask Gurobi to solve a model without specifying the algorithm, it selects what it thinks is the best algorithm to solve each specific problem. Alternatively, we can tell Gurobi to use a specific algorithm to solve the model. In order to do this, we can change the value of the method parameter within the model. The value of this parameter is set to -1 by default, which calls the concurrent optimizer discussed before. For linear programming problems with continuous decision variables, other possible solution algorithms (available with Gurobi) are: primal simplex, dual simplex and barrier algorithm (which is a specific type of interior point method), with method values of 0, 1 and 2 respectively. let’s see how it works in action.

In order to experiment with different algorithms on different problem sizes, we need to organize the code in functions. We explain this step by step below.

We'll investigate the same problem as discussed in [airlines](https://mude.citg.tudelft.nl/book/optimization/airlines.html).

The data we use is a large dataset including all the flights per month in the US for several years for each Origin-Destination pair of cities (from the beginning of 1990 till the end of 2009). We use this information to model a realistic problem (including some simplifying assmptions) that can be modeled as a linear programming problem and solved with Gurobi.

First, let us make a function to read and preprocess the data. It takes the number of month to include as input and returns demand, distance, routes, months and computation time as output. But first things first. We call all packages we need in this tutorial here. Note that we are not using this function yet, we are just defining it here.

In [1]:
from datetime import datetime
import gurobipy as gp
import pandas as pd
import numpy as np

In [2]:
# a function to read and preprocess input data
# it takes the number of month to include as input and returns demand, distance, routes, months and computation time as output
def preprocess(number_of_months):

    # to record execution time
    begin_time = datetime.now()

    # read data
    data = pd.read_csv('flight_edges.tsv', sep='\t', header=None)

    # add column names
    data.columns = ['Origin', 'Destination', 'Origin_City', 'Destination_City', 'Passengers', 'Seats',
                    'Flights', 'Distance', 'Fly_Date', 'Origin_Population', 'Destination_Population']

    # eliminate columns we don't need (for this study)
    data = data.drop(columns=['Origin_City', 'Destination_City', 'Origin_Population', 'Destination_Population'])

    # sort data based on OD and month
    data = data.sort_values(['Origin', 'Destination', 'Fly_Date'])

    # eliminate flights with the same OD (not sure why these records existed, but it makes sense to eliminate them)
    data = data[data['Origin'] != data['Destination']]

    # calculate sum of passengers for each record and add to data (they are spread in multiple lines for some reason)
    data['passenger_sum'] = data.groupby(['Origin', 'Destination', 'Fly_Date'])['Passengers'].transform('sum')
    data['Seats'] = data.groupby(['Origin', 'Destination', 'Fly_Date'])['Seats'].transform('sum')
    data['Flights'] = data.groupby(['Origin', 'Destination', 'Fly_Date'])['Flights'].transform('sum')
    data['Passengers'] = data.groupby(['Origin', 'Destination', 'Fly_Date'])['Passengers'].transform('sum')

    # eliminate duplicates (since aggregated group values, first row in each group is enough to keep
    data = data.drop_duplicates(subset=['Origin', 'Destination', 'Fly_Date'], keep='first')

    # eliminate rows where the number of passengers is less than 5
    data = data[data['passenger_sum'] > 5]

    # some useful variables
    demand_grouped_monthly = data.groupby(['Origin', 'Destination', 'Fly_Date'])['passenger_sum'].agg(np.sum)
    # we have same values for each group, so mean works (as would min, max, etc.)
    distance = data.groupby(['Origin', 'Destination'])['Distance'].mean()
    # note: pandas unique function preserves order (which is the desired behavior here)
    routes = pd.unique(distance.index.values.tolist())
    # note: numpy unique function sorts the output (which is the desired behavior here)
    months = np.unique(data['Fly_Date'])[-1 * number_of_months:]

    # empty dataset with origin-destination pairs (routes) as rows and monthly demand as columns
    demand_data = pd.DataFrame(index=routes, columns=[month for month in months])

    # find monthly demand for each route and add to monthly demand dataset
    # note that iterating through rows of a dataframe can be prohibitively slow, so we use a more efficient way
    for month in months:
        # initiate relevant column with zeros
        demand_data[month].values[:] = 0
        # temporary array to store iteration results
        demand_temp_all_routes = demand_data[month]
        # temporary array to store demand for routes that have none-zero demand values
        demand_temp_nonzero = demand_grouped_monthly[demand_grouped_monthly.index.get_level_values(2) == month]
        # eliminate month from array index (all values here have the same month)
        demand_temp_nonzero.index = demand_temp_nonzero.index.droplevel(2)
        # insert none-zero demand values into relevant rows
        demand_temp_all_routes[demand_temp_all_routes.index.intersection(demand_temp_nonzero.index)] = demand_temp_nonzero
        # assign iteration results (monthly demand) to monthly demand dataset
        demand_data[month] = demand_temp_all_routes

    # convert demand data to a numpy array for efficiency
    demand = demand_data.values

    # computation time
    computation_time = (datetime.now()-begin_time).total_seconds()

    print()
    print('Computation time for data preprocessing with {} periods was: {} seconds'.format(number_of_months, computation_time))
    print()

    return demand, distance, routes, months, computation_time

Next, we need a function to build the model using Gurobi and the process input data from the previous function. It takes demand, distance, routes and months (that we create using preprocess function), and returns the model object and computation time for model construction.

In [3]:
# a function to construct a linear programing model using Gurobi
# it takes demand, distance, routes and months, and returns the model object and computation time for model construction
def model_construct(demand, distance, routes, months):

    # to record execution time
    begin_time = datetime.now()

    # create a gurobi model object
    model = gp.Model()

    # just to avoid unnecessary logging output
    model.Params.LogToConsole = 0

    # define airplane type, capacity, per hour operation cost and purchasing cost and leasing cost
    aircraft_type, aircraft_capacity, aircraft_hourly_operating_cost, aircraft_leasing_cost = gp.multidict({
        'A320-200 CEO': [180, 4378, 196 * 1000],
        'A320 NEO':     [188, 4378, 315 * 1000],
        'A321-200 CEO': [230, 4432, 310 * 1000],
        'A321 NEO':     [236, 5149, 359 * 1000],
        'A330-200':     [436, 6564, 515 * 1000]
    })

    # model parameters
    average_aircraft_speed = 500
    working_day_per_period = 30
    daily_operating_hours = 20

    ## decision variables: frequency Xijm; i: route, j: aircraft type, m: month
    frequency = model.addVars(len(routes), len(aircraft_type), len(months), vtype=gp.GRB.CONTINUOUS, name='frequency')
    fleet_size = model.addVars(len(routes), len(aircraft_type), len(months), vtype=gp.GRB.CONTINUOUS, name='fleet_size')
    model.update()

    ## constraints
    # demand satisfaction:
    # (aircraft capacity) * (route frequency) >= demand
    model.addConstrs(
        gp.quicksum(frequency[i, j, m] * aircraft_capacity[aircraft_type[j]] for j in range(len(aircraft_type)))
        >= demand[i, m]
        for i in range(len(routes))
        for m in range(len(months)))

    # frequency constraint:
    # frequency is restricted by the fleet size (per route, plane type and month)
    # frequency * (flight duration) <= (fleet size) * (available operating hours per period)
    # we do not know flight duration for each route, but we have distance for each route, so time = distance/speed

    model.addConstrs(frequency[i, j, m] * distance[i] / average_aircraft_speed
                     <= fleet_size[i, j, m] * daily_operating_hours * working_day_per_period
                     for i in range(len(routes))
                     for j in range(len(aircraft_type))
                     for m in range(len(months)))

    # set objectives
    # objective function is to minimize total cost, which is the sum of paying for planes and cost of operation:
    # (fleet size) * (aircraft leasing cost) + (frequency) * (flight duration) * (aircraft hourly operating cost)

    obj_leasing_cost = gp.quicksum(fleet_size[i, j, m] * aircraft_leasing_cost[aircraft_type[j]]
                                   for i in range(len(routes))
                                   for j in range(len(aircraft_type))
                                   for m in range(len(months)))

    obj_operating_cost = gp.quicksum(frequency[i, j, m] * aircraft_hourly_operating_cost[aircraft_type[j]] * distance[i] / average_aircraft_speed
                                     for i in range(len(routes))
                                     for j in range(len(aircraft_type))
                                     for m in range(len(months)))

    model.setObjective((obj_leasing_cost + obj_operating_cost), gp.GRB.MINIMIZE)
    model.update()

    # computation time
    computation_time = (datetime.now()-begin_time).total_seconds()

    print()
    print('Computation time for model construction with {} periods was: {} seconds'.format(len(months), computation_time))
    print()

    return model, computation_time

Finally, we need a function to solve the model. It takes the model object we created before and the solution algorithm code as input, and returns the optimal objective function value and the computation time for solving the model. Note that we could decide to return more information with the model (e.g., decision variable values, etc.) but here we are interested in computation times only.

In [4]:
# a function to solve the model
def model_solve(model, algorithm):

    algorithm_names = ['primal simplex', 'dual simplex', 'interior point', 'gurobi default']

    # to record execution time
    begin_time = datetime.now()

    # specify the solution algorithm
    model.Params.Method = algorithm

    # to see the output (remember we turned this off before)
    model.optimize()

    # computation time
    computation_time = (datetime.now()-begin_time).total_seconds()

    print()
    print('Computation time for solving the model using {} method was: {} seconds'.format(algorithm_names[algorithm], computation_time))
    print()

    return model.ObjVal, computation_time

We have almost everything we need. The only thing left is a function to automate running the three functions we created. Here it is.

In [5]:

def run_tasks(number_of_months, algorithm):

    algorithm_names = ['primal simplex', 'dual simplex', 'interior point', 'gurobi default']

    # run preprocessing function
    demand, distance, routes, months, computation_time = preprocess(number_of_months)
    print()
    print('Computation time for data preprocessing with {} periods was: {} seconds'.format(number_of_months, computation_time))
    print()

    # run model construction function
    model, computation_time = model_construct(demand, distance, routes, months)
    print()
    print('Computation time for model construction with {} periods was: {} seconds'.format(number_of_months, computation_time))
    print()

    # solve the model using specified algorithm
    objective, computation_time = model_solve(model, algorithm)
    print()
    print('Computation time for solving the model using {} method was: {} seconds'.format(algorithm_names[algorithm], computation_time))
    print()

    # we could return computation times and objective function values, but we don't need them for now

Now that we have everything we need, we can call these functions with different parameters to observe not only computation times of different algorithms, but also execution times for preprocessing the data and constructing the model (spoiler: you're in for some surprises!).

In [6]:
# define parameters
number_of_months = 6
# 0 for primal simplex and 2 for interior point
algorithm = 0

# run preprocessing function
demand, distance, routes, months, computation_time = preprocess(number_of_months)

# run model construction function
model, computation_time = model_construct(demand, distance, routes, months)

# solve the model using specified algorithm
objective, computation_time = model_solve(model, algorithm)


Computation time for data preprocessing with 6 periods was: 8.119052 seconds

Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-06

Computation time for model construction with 6 periods was: 34.72878 seconds


Computation time for solving the model using primal simplex method was: 0.893729 seconds



In [7]:

# now let's solve with interior point
model.reset()
objective, computation_time = model_solve(model, 2)


Computation time for solving the model using interior point method was: 0.914365 seconds



## Exercise

Run the functions above with different parameter values and make some figures showing how data processing, model construction and computation time (for different algorithms) grow with the number of periods.